This notebook demonstrates the effect of churn and virality on one's playerbase.


In [ ]:
#Import basic math and visualization tools
import numpy as np
import matplotlib.pyplot as pp

In [ ]:
#Parameters
players=10
simulatedDays=100
churnProbability=0.0
viralityProbability=0.1

#placeholder for visualized data
data=np.zeros(simulatedDays)

#simulate
for i in range(simulatedDays):
    data[i]=players
    #some players churn, some really like the game and recruit new players
    players=players - players*churnProbability + players*viralityProbability
    
#plot
pp.plot(data)
pp.xlabel("Days")
pp.ylabel("Players")